In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://apache.osuosl.org/spark/spark-2.2.1/spark-2.2.1-bin-hadoop2.7.tgz
!wget -q https://www-eu.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
# !tar xf https://www-eu.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
# Set environmental variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.2.1-bin-hadoop2.7"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
spark

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('NYC').getOrCreate()

In [0]:
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-04.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-05.csv

--2018-12-09 19:39:39--  https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-04.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.131.101
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.131.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 821249453 (783M) [text/csv]
Saving to: ‘yellow_tripdata_2018-04.csv.2’

yellow_tripdata_201 100%[===================>] 783.20M  44.6MB/s    in 18s     

2018-12-09 19:39:57 (43.0 MB/s) - ‘yellow_tripdata_2018-04.csv.2’ saved [821249453/821249453]

--2018-12-09 19:39:58--  https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-05.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.160.181
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.160.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 814368922 (777M) [text/csv]
Saving to: ‘yellow_tripdata_2018-05.csv.1’

yellow_tripdata_201 100%[===================>] 776.64M  45.5MB/s    in 18s     

2

In [0]:
print(spark.version)

2.4.0


In [0]:
yellow_april = spark.read.csv("yellow_tripdata_2018-04.csv", header=True, inferSchema=True)
yellow_may = spark.read.csv("yellow_tripdata_2018-05.csv", header=True, inferSchema=True)

In [0]:
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2018-04.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2018-05.csv
  

--2018-12-09 19:47:22--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2018-04.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.102.85
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.102.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71636786 (68M) [text/csv]
Saving to: ‘green_tripdata_2018-04.csv’

green_tripdata_2018 100%[===================>]  68.32M  35.3MB/s    in 1.9s    

2018-12-09 19:47:24 (35.3 MB/s) - ‘green_tripdata_2018-04.csv’ saved [71636786/71636786]

--2018-12-09 19:47:25--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2018-05.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.102.85
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.102.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71376136 (68M) [text/csv]
Saving to: ‘green_tripdata_2018-05.csv’

green_tripdata_2018 100%[===================>]  68.07M  35.2MB/s    in 1.9s    

2018-12-09 19:47:27 (3

In [0]:
green_april = spark.read.csv("green_tripdata_2018-04.csv", header=True, inferSchema=True)
green_may = spark.read.csv("green_tripdata_2018-05.csv", header=True, inferSchema=True)

In [0]:
yellow_combined = yellow_april.union(yellow_may) #<- combined dataset

In [0]:
green_combined = green_april.union(green_may)  #<- combined green data

In [0]:
# check summary
yellow_combined.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2018-04-01 00:22:20|  2018-04-01 00:22:26|              1|          0.0|         1|                 N|         145|         145|           2|        2.5|  0.5|    0.5|       0.0|         0.0|                  0.3|         3.8|
|       1| 2018-04-01 00:47:37|  2018-04-01 01:0

In [0]:
yellow_combined.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [0]:
green_combined.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+
|       2| 2018-04-01 00:55:44|  2018-04-01 01:01:33|                 N|         1|          80|         256|              1|         0.91|        6.0|  0.5|    0.5|       0.0|         0.0|     null|                  0.3|     

In [0]:
green_combined.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)



In [0]:
yellow_combined.describe().show()

+-------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+--------------------+------------------+------------------+---------------------+-----------------+
|summary|           VendorID|   passenger_count|     trip_distance|        RatecodeID|store_and_fwd_flag|      PULocationID|      DOLocationID|      payment_type|       fare_amount|              extra|             mta_tax|        tip_amount|      tolls_amount|improvement_surcharge|     total_amount|
+-------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+--------------------+------------------+------------------+---------------------+-----------------+
|  count|           18529578|          18529578|          18529578|          18529578|          1

In [0]:
green_combined.describe().show()

+-------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+---------+---------------------+------------------+------------------+-------------------+
|summary|           VendorID|store_and_fwd_flag|        RatecodeID|      PULocationID|      DOLocationID|   passenger_count|     trip_distance|       fare_amount|             extra|            mta_tax|        tip_amount|       tolls_amount|ehail_fee|improvement_surcharge|      total_amount|      payment_type|          trip_type|
+-------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+---------+---------------------+------------------+------------------+-------------------+
|  coun

In [0]:
print((yellow_combined.count(), len(yellow_combined.columns)))

(18529578, 17)


In [0]:
print((green_combined.count(), len(green_combined.columns)))

(1597317, 19)


### VendorID

4 - is junk - remove

In [0]:
yellow_combined.groupBy('VendorID').count().show()

+--------+--------+
|VendorID|   count|
+--------+--------+
|       1| 8046272|
|       4|    1130|
|       2|10482176|
+--------+--------+



In [0]:
yellow_combined = yellow_combined.filter(yellow_combined["VendorID"]!=4)

In [0]:
green_combined.groupBy('VendorID').count().show()

+--------+-------+
|VendorID|  count|
+--------+-------+
|       1| 268393|
|       2|1328924|
+--------+-------+



### store_and_fwd_flag

In [0]:
green_combined.groupBy('store_and_fwd_flag').count().show()

+------------------+-------+
|store_and_fwd_flag|  count|
+------------------+-------+
|                 Y|   2327|
|                 N|1594990|
+------------------+-------+



In [0]:
yellow_combined.groupBy('store_and_fwd_flag').count().show()

+------------------+--------+
|store_and_fwd_flag|   count|
+------------------+--------+
|                 Y|   64885|
|                 N|18464693|
+------------------+--------+



### passenger_count

In [0]:
yellow_combined.groupBy('passenger_count').count().show()

+---------------+--------+
|passenger_count|   count|
+---------------+--------+
|              1|13201488|
|              6|  505987|
|              3|  756841|
|              5|  852545|
|              9|      59|
|              4|  356618|
|              8|      65|
|              7|      67|
|              2| 2710998|
|              0|  144910|
+---------------+--------+



In [0]:
green_combined.groupBy('passenger_count').count().show()

+---------------+-------+
|passenger_count|  count|
+---------------+-------+
|              1|1345627|
|              6|  30109|
|              3|  27655|
|              5|  54726|
|              9|      5|
|              4|   9443|
|              8|     14|
|              7|     21|
|              2| 126936|
|              0|   2781|
+---------------+-------+



In [0]:
# Zero passenger analysis
zero_passenger = yellow_combined.filter(yellow_combined["passenger_count"]==0)

In [0]:
zero_passenger.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2018-04-01 00:20:09|  2018-04-01 00:24:44|              0|          0.8|         1|                 N|          50|          48|           1|        5.0|  0.5|    0.5|       1.0|         0.0|                  0.3|         7.3|
|       1| 2018-04-01 00:43:37|  2018-04-01 00:5

In [0]:
zero_passenger.describe().show()

+-------+-------------------+---------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+------------------+---------------------+------------------+
|summary|           VendorID|passenger_count|     trip_distance|        RatecodeID|store_and_fwd_flag|      PULocationID|     DOLocationID|      payment_type|       fare_amount|              extra|            mta_tax|        tip_amount|      tolls_amount|improvement_surcharge|      total_amount|
+-------+-------------------+---------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+------------------+---------------------+------------------+
|  count|             144910|         144910|            144910|            144910|            144910|       

In [0]:
yellow_combined = yellow_combined.filter(yellow_combined["passenger_count"]!=0)

In [0]:
green_combined = green_combined.filter(green_combined["passenger_count"]!=0)

### Finding outliers in numerical columns - yellow_combined

In [0]:
cols = ['trip_distance','fare_amount', 'extra', 'mta_tax', 'tip_amount', 
        'tolls_amount', 'improvement_surcharge', 'total_amount']
bounds = {}
for col in cols:
  quantiles = yellow_combined.approxQuantile(
      col, [0.25, 0.75], 0.05
  )
  IQR = quantiles[1] - quantiles[0]
  bounds[col] = [
      quantiles[0] - 1.5 * IQR,
      quantiles[1] + 1.5 * IQR
  ]

In [0]:
bounds

{'extra': [-1.5, 2.5],
 'fare_amount': [-5.25, 28.75],
 'improvement_surcharge': [0.3, 0.3],
 'mta_tax': [0.5, 0.5],
 'tip_amount': [-4.275, 7.125],
 'tolls_amount': [0.0, 0.0],
 'total_amount': [-5.104999999999999, 33.975],
 'trip_distance': [-3.2649999999999992, 8.375]}

### Trip distance

remove trip_distance less than 0
remove upper bound outliers

In [0]:
yellow_combined.filter(yellow_combined["trip_distance"] > bounds['trip_distance'][1]).count()

1530107

In [0]:
yellow_combined = yellow_combined.filter((yellow_combined["trip_distance"]  > 0) & 
                                         (yellow_combined["trip_distance"] < bounds['trip_distance'][1]))

In [0]:
print((yellow_combined.count(), len(yellow_combined.columns)))

(16738744, 17)


### Fare amount
min is 2.5
remove outlier from upper bound

In [0]:
yellow_combined.filter((yellow_combined["fare_amount"]  > 2.5) & 
                                         (yellow_combined["fare_amount"] < bounds['fare_amount'][1])).count()

16562037

In [0]:
yellow_combined = yellow_combined.filter((yellow_combined["fare_amount"]  > 2.5) & 
                                         (yellow_combined["fare_amount"] < bounds['fare_amount'][1]))

In [0]:
print((yellow_combined.count(), len(yellow_combined.columns)))

(16562037, 17)


In [0]:
yellow_combined.describe().show()

+-------+------------------+------------------+------------------+-------------------+------------------+-----------------+-----------------+-------------------+------------------+-------------------+--------------------+------------------+--------------------+---------------------+------------------+
|summary|          VendorID|   passenger_count|     trip_distance|         RatecodeID|store_and_fwd_flag|     PULocationID|     DOLocationID|       payment_type|       fare_amount|              extra|             mta_tax|        tip_amount|        tolls_amount|improvement_surcharge|      total_amount|
+-------+------------------+------------------+------------------+-------------------+------------------+-----------------+-----------------+-------------------+------------------+-------------------+--------------------+------------------+--------------------+---------------------+------------------+
|  count|          16562037|          16562037|          16562037|           16562037|     

### Green_combined - remove outliers in numerical data

In [0]:
cols = ['trip_distance','fare_amount', 'extra', 'mta_tax', 'tip_amount', 
        'tolls_amount', 'improvement_surcharge', 'total_amount']
green_bounds = {}
for col in cols:
  quantiles = green_combined.approxQuantile(
      col, [0.25, 0.75], 0.05
  )
  IQR = quantiles[1] - quantiles[0]
  green_bounds[col] = [
      quantiles[0] - 1.5 * IQR,
      quantiles[1] + 1.5 * IQR
  ]

In [0]:
green_bounds

{'extra': [-1.5, 2.5],
 'fare_amount': [-6.0, 30.0],
 'improvement_surcharge': [0.3, 0.3],
 'mta_tax': [0.5, 0.5],
 'tip_amount': [-2.4899999999999998, 4.1499999999999995],
 'tolls_amount': [0.0, 0.0],
 'total_amount': [-11.95, 42.05],
 'trip_distance': [-3.9999999999999996, 9.6]}

### Trip distance
 remove less than 0
 remove upper bound outliers

In [0]:
green_combined = green_combined.filter((green_combined["trip_distance"]  > 0) & 
                                         (green_combined["trip_distance"] < green_bounds['trip_distance'][1]))

In [0]:
print((green_combined.count(), len(green_combined.columns)))

(1486174, 19)


### Fare Amount

remove less than 2.5
remove upper bound outliers

In [0]:
green_combined = green_combined.filter((green_combined["fare_amount"]  > 2.5) & 
                                         (green_combined["fare_amount"] < green_bounds['fare_amount'][1]))

In [0]:
print((green_combined.count(), len(green_combined.columns)))

(1440044, 19)


### RateCodeID

In [0]:
yellow_combined.groupBy('RateCodeID').count().show()

+----------+--------+
|RateCodeID|   count|
+----------+--------+
|         1|16557633|
|         6|      11|
|         3|    1280|
|         5|    1925|
|         4|    1176|
|         2|       5|
|        99|       7|
+----------+--------+



In [0]:
yellow_combined = yellow_combined.filter(yellow_combined["RateCodeID"]!= 99)

In [0]:
green_combined.groupBy('RateCodeID').count().show()

+----------+-------+
|RateCodeID|  count|
+----------+-------+
|         1|1427391|
|         6|      3|
|         3|    118|
|         5|  12346|
|         4|    186|
+----------+-------+



### Payment_type

In [0]:
yellow_combined.groupBy('payment_type').count().show()

+------------+--------+
|payment_type|   count|
+------------+--------+
|           1|11445519|
|           3|   62829|
|           4|   16909|
|           2| 5036773|
+------------+--------+



In [0]:
green_combined.groupBy('payment_type').count().show()

+------------+------+
|payment_type| count|
+------------+------+
|           1|789503|
|           3|  3218|
|           5|    42|
|           4|  2484|
|           2|644797|
+------------+------+



### Trip type - only in green

In [0]:
green_combined.groupBy('trip_type').count().show()

+---------+-------+
|trip_type|  count|
+---------+-------+
|        1|1426096|
|        2|  13948|
+---------+-------+



In [0]:
print((green_combined.count(), len(green_combined.columns)))

(1440044, 19)


In [0]:
print((yellow_combined.count(), len(yellow_combined.columns)))

(16562030, 17)


### Time component - additional features

In [0]:
from pyspark.sql import functions as F
timeFmt = "yyyy-MM-dd'T'HH:mm:ss.SSS"
timeDiff = (F.unix_timestamp('tpep_dropoff_datetime', format=timeFmt)
            - F.unix_timestamp('tpep_pickup_datetime', format=timeFmt))
yellow_combined = yellow_combined.withColumn("duration", timeDiff/60)

In [0]:
cols = ['duration']
time_bounds = {}
for col in cols:
  quantiles = yellow_combined.approxQuantile(
      col, [0.25, 0.75], 0.05
  )
  IQR = quantiles[1] - quantiles[0]
  time_bounds[col] = [
      quantiles[0] - 1.5 * IQR,
      quantiles[1] + 1.5 * IQR
  ]
  
time_bounds

{'duration': [-9.166666666666668, 35.1]}

In [0]:
yellow_combined = yellow_combined.filter((yellow_combined["duration"]  > 0) & (yellow_combined["duration"] < time_bounds['duration'][1]))

In [0]:
from pyspark.sql import functions as F
timeFmt = "yyyy-MM-dd'T'HH:mm:ss.SSS"
timeDiff = (F.unix_timestamp('lpep_dropoff_datetime', format=timeFmt)
            - F.unix_timestamp('lpep_pickup_datetime', format=timeFmt))
green_combined = green_combined.withColumn("duration", timeDiff/60)

In [0]:
cols = ['duration']
time_bounds = {}
for col in cols:
  quantiles = green_combined.approxQuantile(
      col, [0.25, 0.75], 0.05
  )
  IQR = quantiles[1] - quantiles[0]
  time_bounds[col] = [
      quantiles[0] - 1.5 * IQR,
      quantiles[1] + 1.5 * IQR
  ]
  
time_bounds

{'duration': [-9.15, 31.516666666666666]}

In [0]:
green_combined = green_combined.filter((green_combined["duration"]  > 0) & (green_combined["duration"] < time_bounds['duration'][1]))

In [0]:
print((green_combined.count(), len(green_combined.columns)))
print((yellow_combined.count(), len(yellow_combined.columns)))

(1380412, 20)
(16344000, 18)


### Date columns

In [0]:
from pyspark.sql.functions import date_format
from pyspark.sql.functions import isnan, when, count, col

date_green = green_combined.select('lpep_pickup_datetime', 
                                   date_format('lpep_pickup_datetime', 'H').alias('hour'), 
                                   date_format('lpep_pickup_datetime', 'd').alias('date'), 
                                   date_format('lpep_pickup_datetime', 'u').alias('day_number'), date_format('lpep_pickup_datetime', 'E').alias('day'))
#green.show()
date_green = date_green.withColumn('is_weekend', when((date_green.day == 'Sun') | (date_green.day == 'Sat'),1).otherwise(0))
# Convert string to numeric

from pyspark.sql.types import IntegerType
date_green = date_green.withColumn("hour", date_green["hour"].cast(IntegerType()))
date_green = date_green.withColumn("date", date_green["date"].cast(IntegerType()))
date_green = date_green.withColumn("day_number", date_green["day_number"].cast(IntegerType()))
date_green = date_green.withColumn("is_weekend", date_green["is_weekend"].cast(IntegerType()))

In [0]:
date_yellow = yellow_combined.select('tpep_pickup_datetime', 
                                     date_format('tpep_pickup_datetime', 'H').alias('hour'), 
                                     date_format('tpep_pickup_datetime', 'd').alias('date_yellow'), 
                                     date_format('tpep_pickup_datetime', 'u').alias('day_number'), date_format('tpep_pickup_datetime', 'E').alias('day'))
#yellow_combined.show()
date_yellow = date_yellow.withColumn('is_weekend',when((date_yellow.day == 'Sun') | (date_yellow.day == 'Sat'),1).otherwise(0))
# Convert string to numeric

from pyspark.sql.types import IntegerType
date_yellow = date_yellow.withColumn("hour", date_yellow["hour"].cast(IntegerType()))
date_yellow = date_yellow.withColumn("date_yellow", date_yellow["date_yellow"].cast(IntegerType()))
date_yellow = date_yellow.withColumn("day_number", date_yellow["day_number"].cast(IntegerType()))
date_yellow = date_yellow.withColumn("is_weekend", date_yellow["is_weekend"].cast(IntegerType()))

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
green_combined = green_combined.drop("lpep_pickup_datetime")
green_combined = green_combined.withColumn("id", monotonically_increasing_id())

date_green = date_green.withColumn("id", monotonically_increasing_id())


green_combined = date_green.join(green_combined, "id", "outer").drop("id")

green_combined.show()

+--------------------+----+----+----------+---+----------+--------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+------------------+
|lpep_pickup_datetime|hour|date|day_number|day|is_weekend|VendorID|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|          duration|
+--------------------+----+----+----------+---+----------+--------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+------------------+
| 2018-04-01 00:50:49|   0|   1|         7|Sun|         1|   

In [0]:
print((yellow_combined.count(), len(yellow_combined.columns)))
print((date_yellow.count(), len(date_yellow.columns)))

(16344000, 18)
(16344000, 6)


In [0]:
from pyspark.sql.functions import monotonically_increasing_id
yellow_combined = yellow_combined.drop("tpep_pickup_datetime")
yellow_combined = yellow_combined.withColumn("id", monotonically_increasing_id())

date_yellow = date_yellow.withColumn("id", monotonically_increasing_id())


yellow_combined = date_yellow.join(yellow_combined, "id", "outer").drop("id")

yellow_combined.show(5)

+--------------------+----+-----------+----------+---+----------+--------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------------+
|tpep_pickup_datetime|hour|date_yellow|day_number|day|is_weekend|VendorID|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|          duration|
+--------------------+----+-----------+----------+---+----------+--------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------------+
| 2018-04-01 00:42:17|   0|          1|         7|Sun|         1|       1|  2018-04-01 00:44:13|    

In [0]:
yellow_combined = yellow_combined.withColumn("speed", yellow_combined['trip_distance']*60/yellow_combined['duration'])

In [0]:
yellow_combined.show(5)

+--------------------+----+-----------+----------+---+----------+--------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------------+------------------+
|tpep_pickup_datetime|hour|date_yellow|day_number|day|is_weekend|VendorID|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|          duration|             speed|
+--------------------+----+-----------+----------+---+----------+--------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------------+------------------+
| 2018-04-01 00:42:17|   0|          1|    

In [0]:
green_combined = green_combined.withColumn("speed", green_combined['trip_distance']*60/green_combined['duration'])

In [0]:
print((yellow_combined.count(), len(yellow_combined.columns)))
print((green_combined.count(), len(green_combined.columns)))

(16344000, 24)
(1380412, 26)


In [0]:
yellow_combined = yellow_combined.drop("tpep_pickup_datetime", "tpep_dropoff_datetime")
from pyspark.sql.functions import isnan, when, count, col
yellow_combined.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in yellow_combined.columns]).show(5)

+----+-----------+----------+---+----------+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------+-----+
|hour|date_yellow|day_number|day|is_weekend|VendorID|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|duration|speed|
+----+-----------+----------+---+----------+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------+-----+
|   0|          0|         0|  0|         0|       0|              0|            0|         0|                 0|           0|           0|           0|          0|    0|      0|         0|           0|                    0|           0|    

In [0]:
green_combined = green_combined.drop("lpep_pickup_datetime", "lpep_dropoff_datetime")
from pyspark.sql.functions import isnan, when, count, col
green_combined.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in green_combined.columns]).show(5)

+----+----+----------+---+----------+--------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------+-----+
|hour|date|day_number|day|is_weekend|VendorID|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|duration|speed|
+----+----+----------+---+----------+--------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------+-----+
|   0|   0|         0|  0|         0|       0|                 0|         0|           0|           0|              0|            0|          0|    0|      0|         0|           0|  1380412|          